In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

In [2]:
def users_to_json(spark_context, file):
    def json_lambda(line):
        json_line = {}
        tags_list={'Id':int,
                   'Reputation':int,
                   'CreationDate':str,
                   'DisplayName':str,
                   'LastAccessDate':str,
                   'WebsiteUrl':str,
                   'Location':str,
                   'AboutMe':str,
                   'Views':int,
                   'UpVotes':int,
                   'DownVotes':int,
                   'EmailHash':str,
                   'AccountId':int,
                   'Age':float}
        indices = [line.find(tag,0) for tag in tags_list if line.find(tag,0)!=-1]
        for i in range(len(indices)-1):
            parsed = line[indices[i]:indices[i+1]].split('=')
            if parsed[0] == 'Id':
                try:
                    json_line[parsed[0]] = tags_list[parsed[0]](parsed[1])
                except ValueError:
                    json_line[parsed[0]] = tags_list[parsed[0]](parsed[1][1:-2])
        return json_line
    usersRDD = spark_context.textFile(file)
    headers = usersRDD.take(2)
    usersRDD = usersRDD.filter(lambda x: x not in  headers)
    usersRDD = usersRDD.map(json_lambda)
    return usersRDD



In [9]:
import re
data_path = '/home/piotr/big_data/archive.org/download/stackexchange/'
import os
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

subdirs = get_immediate_subdirectories(data_path)
for subdir in subdirs:
    if not re.search('meta', subdir):
        try:
            RDD_json = users_to_json(spark.sparkContext, data_path + subdir + '/Users.xml')
            name = subdir.split('.')[0]
            RDD_json.saveAsTextFile('./jsons/' + name)
        except:
            print(name)
        

aviation
conlang
quantumcomputing
